In [ ]:
import os, sys
import time
import pickle
import gzip
from typing import *

from pyk4a import PyK4A, Config
from matplotlib import pyplot as plt
import numpy as np

def save_gzip(path, data):
    with gzip.open(path, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
device_id = 0
config = {}


# Load camera with the default config
config = Config(**config)
print(config.unpack())


camera = PyK4A(device_id=device_id, config=config)
camera.start()
time.sleep(0.5)

In [ ]:
fps = 10
N_frames = 100
use_depth_frame = False

captures = []
if camera.is_running:
    for _ in range(N_frames):
        capture = camera.get_capture()
        if capture is not None:
            if use_depth_frame:
                color, depth = capture.transformed_color, capture.depth
            else:
                color, depth = capture.color, capture.transformed_depth
            color = color[:, :, 2::-1]

            captures.append((color,depth))
        else:
            print("Empty frame observed!")
        time.sleep(1/fps)

In [ ]:
save_dir = 'recorded'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

save_gzip(os.path.join(save_dir, 'captures.gzip'), captures)
camera.save_calibration_json(os.path.join(save_dir,'calibrations.json'))